# Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


# Optimization_1 : Optimizing by dropping more or fewer columns

In [3]:
# Check value count of columns of 'STATUS' and 'SPECIAL_CONSIDERATIONS'
display(application_df['STATUS'].value_counts())
display(application_df['SPECIAL_CONSIDERATIONS'].value_counts())

1    34294
0        5
Name: STATUS, dtype: int64

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [4]:
# Drop the '0' records from 'STATUS', which is considered as inactive applications. And drop the 'Y' records from 'SPECIAL_CONSIDERATIONS', which is considered as special cases.
# After droping those outliner applications, we can drop the whole 'STATUS' and 'SPECIAL_CONSIDERATIONS' columns, since there is only 1 value in each column.
application_df_reduce_col = application_df.loc[(application_df['STATUS']!=0) & (application_df['SPECIAL_CONSIDERATIONS']!='Y')].drop(columns=['STATUS','SPECIAL_CONSIDERATIONS'])
application_df_reduce_col

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [5]:
#Remain the same binning for 'APPLICATION_TYPE' and 'CLASSIFICATION'
#-------------------------------------------------------------------------------------------------------------------------------------------------
application_types_to_replace = application_df_reduce_col['APPLICATION_TYPE'].value_counts().loc[application_df_reduce_col['APPLICATION_TYPE'].value_counts()<528].keys()
# Replace in dataframe
for app in application_types_to_replace:
    application_df_reduce_col['APPLICATION_TYPE'] = application_df_reduce_col['APPLICATION_TYPE'].replace(app,"Other")
# Check to make sure binning was successful
display(application_df_reduce_col['APPLICATION_TYPE'].value_counts())
#-------------------------------------------------------------------------------------------------------------------------------------------------
classifications_to_replace = application_df_reduce_col['CLASSIFICATION'].value_counts().loc[application_df_reduce_col['CLASSIFICATION'].value_counts()<=777].keys()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df_reduce_col['CLASSIFICATION'] = application_df_reduce_col['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
display(application_df_reduce_col['CLASSIFICATION'].value_counts())


T3       27006
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

C1000    17307
C2000     6069
C1200     4833
Other     2259
C3000     1918
C2100     1882
Name: CLASSIFICATION, dtype: int64

In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_reduce_col_dummies = pd.get_dummies(application_df_reduce_col[['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE',	'ORGANIZATION',	'INCOME_AMT']])
application_df_reduce_col_numeric_columns = application_df_reduce_col[['ASK_AMT','IS_SUCCESSFUL']]
application_df_reduce_col_afterDummies = pd.concat([application_df_reduce_col_dummies,application_df_reduce_col_numeric_columns],axis=1)
application_df_reduce_col_afterDummies

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,ASK_AMT,IS_SUCCESSFUL
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5000,1
1,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,108590,1
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,5000,0
3,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,6692,1
4,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5000,0
34295,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,5000,0
34296,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,5000,0
34297,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,5000,1


In [7]:
# Split our preprocessed data into our features and target arrays
# y represent Target, X represent Features
y = application_df_reduce_col_afterDummies['IS_SUCCESSFUL']
X = application_df_reduce_col_afterDummies.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=40))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3280      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5741 (22.43 KB)
Trainable params: 5741 (22.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 8s 6ms/step - loss: 0.5707 - accuracy: 0.7203
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5559 - accuracy: 0.7278
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5532 - 

# Summary:
Performance: Loss: 0.5586516857147217, Accuracy: 0.7303606867790222

Dropping more or fewer columns: `'STATUS'` and `'SPECIAL_CONSIDERATIONS'` does not increase the accuracy.
So we try to create more bins for rare occurrences in columns.

# Optimization_2 : Optimizing by creating more bins for rare occurrences in columns

In [30]:
# load csv and drop the non-beneficial ID columns, 'EIN' and 'NAME' again
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = application_df.drop(columns=['EIN','NAME'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [31]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts().loc[application_df['APPLICATION_TYPE'].value_counts()<100].keys()
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
# Check to make sure binning was successful
display(application_df['APPLICATION_TYPE'].value_counts())
#----------------------------------------------------------------------------------------------------------------------------------------------------
# Look at CLASSIFICATION value counts for binning
CLASSIFICATION_value_count = application_df['CLASSIFICATION'].value_counts()
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = CLASSIFICATION_value_count.loc[CLASSIFICATION_value_count<=100].keys()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
# Check to make sure binning was successful
display(application_df['CLASSIFICATION'].value_counts())

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [32]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df[['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE',	'ORGANIZATION',	'INCOME_AMT','SPECIAL_CONSIDERATIONS']])
numeric_columns = application_df[['STATUS','ASK_AMT','IS_SUCCESSFUL']]
application_df_afterDummies = pd.concat([application_df_dummies,numeric_columns],axis=1)
application_df_afterDummies

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT,IS_SUCCESSFUL
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,5000,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,108590,1
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,5000,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,6692,1
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,5000,0
34295,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,5000,0
34296,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,5000,0
34297,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,5000,1


In [33]:
# Split our preprocessed data into our features and target arrays
# y represent Target, X represent Features
y = application_df_afterDummies['IS_SUCCESSFUL']
X = application_df_afterDummies.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)

In [34]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=50))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                4080      
                                                                 
 dense_4 (Dense)             (None, 60)                4860      
                                                                 
 dense_5 (Dense)             (None, 1)                 61        
                                                                 
Total params: 9001 (35.16 KB)
Trainable params: 9001 (35.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5691 - accuracy: 0.7211
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5543 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5509 - accuracy: 0.7293
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7318
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7314
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5454 - accuracy: 0.7308
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accuracy: 0.7346
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5444 - accuracy: 0.7329
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5433 - accuracy: 0.7334
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5421 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5663 - accuracy: 0.7273 - 549ms/epoch - 2ms/step
Loss: 0.5662509202957153, Accuracy: 0.7273469567298889


# Summary:
Performance: Loss: 0.5662509202957153, Accuracy: 0.7273469567298889

Creating more bins for rare occurrences in columns: `'APPLICATION_TYPE'` and `'CLASSIFICATION'` does not increase the accuracy.
So we try to add more hidden layers.

# Optimization_3 : Optimizing by adding one more hidden layer

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=160, activation="relu", input_dim=50))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=160, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=160, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 160)               8160      
                                                                 
 dense_19 (Dense)            (None, 160)               25760     
                                                                 
 dense_20 (Dense)            (None, 160)               25760     
                                                                 
 dense_21 (Dense)            (None, 1)                 161       
                                                                 
Total params: 59841 (233.75 KB)
Trainable params: 59841 (233.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5667 - accuracy: 0.7242
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5536 - accuracy: 0.7290
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5499 - accuracy: 0.7307
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5475 - accuracy: 0.7317
Epoch 5/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5466 - accuracy: 0.7326
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5448 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5443 - accuracy: 0.7334
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5432 - accuracy: 0.7346
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5429 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5421 - accura

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5800 - accuracy: 0.7278 - 787ms/epoch - 3ms/step
Loss: 0.5799769163131714, Accuracy: 0.7278134226799011


# Summary:
Performance: Loss: 0.5799769163131714, Accuracy: 0.7278134226799011

Adding one more hidden layer does not increase the accuracy.

# Try to use AutoOptimization to find out a better solution

In [38]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=50))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [39]:
# Import the kerastuner library
!pip install keras-tuner
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [40]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 56 Complete [00h 01m 24s]
val_accuracy: 0.7209329605102539

Best val_accuracy So Far: 0.7315452098846436
Total elapsed time: 00h 26m 57s


## Summary:
Best val_accuracy So Far: 0.7315452098846436

So we will need to do more trainning or adding more data to achieve the targe accuracy of 75%.